## 텍스트 클렌징

### 라이브러리 로딩, 형태소 분석기(Komoran) 초기화, 데이터 불러오기

In [11]:
import pandas as pd
import re
import os
import xlsxwriter
from PyKomoran import *
from tqdm import tqdm # 프로그레스바 출력을 위한 라이브러리


INPUT_FILE_PATH = "./output/03_data_output.xlsx" # 입력 데이터 파일
OUPUT_PATH = "./output/" # 데이터 출력 경로


komoran = Komoran("EXP") # komoran full version 모델 적용
komoran.set_user_dic("./user_dic/dic1.user") # 사용자 사전 등록

komoran_light = Komoran("STABLE") # komoran light version 모델 적용

dataframe = pd.read_excel(INPUT_FILE_PATH) # 데이터 가져오기

### 불용어 단어사전 불러오기

In [12]:
stopwords = set()
with open("./user_dic/stopwords", "r") as f:
    for line in f.readlines():
        stopwords.add(line.strip())

### 명사 추출

In [13]:
extracted_nouns = [] # 각 판례들에 대한 명사들 저장
filtered_judgement_reason = [] # 필터링된 판결 이유 저장

for reason in tqdm(dataframe["judgement_reason"]):
    # 텍스트 클렌징
    reason = re.sub(r'\d+?\.\s\d+\.\s\d+\.', '', reason) # 날짜 제거
    reason = re.sub(r'\b\w+법원|지원', '', reason) # 법원명 제거
    reason = re.sub('수사보고|범 죄 사 실|범죄사실', '', reason) # 판례의 기본적인 문구 제거
    reason = re.sub(r'[「」『』\[\],.:%○]', '', reason) # 특수기호 제거

    # Full 명사 추출
    nouns = []
    for noun in komoran.get_nouns(reason):
        if noun.isalpha() and (noun not in stopwords) and (len(noun) > 1): # 단어에 숫자가 포함되지 않는 경우에만 저장 and 불용어 사전에 포함되지 않는 경우 and 단어 길이가 2이상인 단어만 저장
            nouns.append(noun.replace(" ", "")) # 띄어쓰기를 제거한 후 저장

    # Light 명사 추출
    for noun in komoran_light.get_nouns(reason):
      if noun.isalpha() and (noun not in stopwords) \
          and (len(noun) > 1):
          nouns.append(noun.replace(" ", "")) # 띄어쓰기 제거

    extracted_nouns.append(' '.join(nouns))
    filtered_judgement_reason.append(reason)

dataframe['extracted_nouns'] = extracted_nouns
dataframe['filtered_judgement_reason'] = filtered_judgement_reason

### 데이터 저장

In [10]:
dataframe.to_excel(os.path.join(OUPUT_PATH, "04_data_output.xlsx"), index=False, engine = "xlsxwriter")

ValueError: Length of values does not match length of index

### 전체 데이터에서 가장 많이 사용된 명사 단어 추출
불용어 사전을 만들기 위해 전체 데이터에서 가장 많이 사용된 명사를 내림차순으로 출력합니다.

In [101]:
#########################
# 전체 데이터에서 가장 많이 사용된 명사 단어 추출
#########################
word_counter = {}
for reason in dataframe['judgement_reason']:
    # 텍스트 클렌징
    reason = re.sub(r'\d+?\.\s\d+\.\s\d+\.', '', reason) # 날짜 제거
    reason = re.sub(r'\b\w+법원|지원', '', reason) # 법원명 제거
    reason = re.sub('\n[0-9]. ', '', reason) # 목차 숫자 제거 1
    reason = re.sub('\n\([0-9]\) ', '', reason) # 목차 숫자 제거 2
    reason = re.sub('\n[가-하]. ', '', reason) # 목차 한글 제거
    reason = re.sub('수사보고|범 죄 사 실|범죄사실', '', reason) # 판례의 기본적인 문구 제거

    # 명사 추출
    nouns = komoran.get_nouns(reason) # 명사 추출
    nouns = list(filter(lambda x : len(x) > 1, nouns)) # 단어 길이가 2 이상인 단어만 추출

    # 워드 카운터
    for noun in nouns:
        if noun not in word_counter.keys():
            word_counter[noun] = 1
        else:
            word_counter[noun] += 1

print(sorted(word_counter.items(), key=lambda x:x[1], reverse=True)[:300])

[('피고인', 97917), ('운전', 38618), ('음주운전', 34030), ('형법', 32611), ('범행', 27101), ('차량', 25348), ('피해자', 24043), ('승용차', 21369), ('양형', 20810), ('상태', 20719), ('사건', 19946), ('도로', 19326), ('선택', 18043), ('정상', 16907), ('조의', 16748), ('정황', 14082), ('도로교통법', 13699), ('혈중알코올농도', 13646), ('서울', 13417), ('처벌', 13369), ('번호', 12944), ('생략', 12831), ('주취운전자', 12464), ('증거', 12292), ('이유', 11994), ('적용', 11212), ('해당', 11208), ('전력', 10957), ('요지', 10944), ('법정진술', 10882), ('징역형', 10718), ('집행유예', 10603), ('법조', 10521), ('법령', 10478), ('교통사고', 10066), ('자동차', 9866), ('사고', 9671), ('참작', 9146), ('주문', 9036), ('구간', 8745), ('결과', 8471), ('진행', 8380), ('상해', 8109), ('사람', 8083), ('범죄', 7995), ('성행', 7828), ('벌금형', 7748), ('정황진술보고서', 7705), ('위반', 7654), ('약식명령', 7572), ('종합', 7342), ('조건', 6968), ('부분', 6948), ('인정', 6883), ('종사', 6653), ('벌금', 6547), ('차로', 6476), ('명령', 6446), ('필요', 6347), ('환경', 6346), ('전방', 5997), ('범죄전력', 5888), ('조회', 5850), ('동기', 5638), ('변론', 5635), ('발생', 5631), ('치료',